<a href="https://colab.research.google.com/github/alanalqobaisi/awget-11-14/blob/main/Project1AlanAlqobaisi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#!/usr/bin/python3
from networkx import barabasi_albert_graph
from csv import writer


def main():
    # create barabasi albert network with 4000, 7000, and 10000 nodes
    # and (10%) node degree
    filenames = ['barabasi_albert4000.csv', 'barabasi_albert7000.csv', 'barabasi_albert10000.csv']
    networksizes = [4000, 7000, 10000]

    for idx, filename in enumerate(filenames):
        graph = barabasi_albert_graph(networksizes[idx], int(networksizes[idx] * 0.10))
        try:
            with open(filename, 'w', newline='') as outfile:
                csvwriter = writer(outfile)
                for edge in graph.edges:
                    csvwriter.writerow(edge)
        except IOError as e:
            print("Unable to open or write " + filename + " due to Error: " + e.strerror)
            exit(0)


if __name__ == "__main__":
    main()


In [5]:

from networkx import erdos_renyi_graph
from csv import writer


def main():
    # create erdos renyi network with 4000, 7000, and 10000 nodes
    # and (0.10) probability of edges creation
    filenames = ['erdos_renyi4000.csv', 'erdos_renyi7000.csv', 'erdos_renyi10000.csv']
    networksizes = [4000, 7000, 10000]

    for idx, filename in enumerate(filenames):
        graph = erdos_renyi_graph(networksizes[idx], 0.10)
        try:
            with open(filename, 'w', newline='') as outfile:
                csvwriter = writer(outfile)
                for edge in graph.edges:
                    csvwriter.writerow(edge)
        except IOError as e:
            print("Unable to open or write " + filename + " due to Error: " + e.strerror)
            exit(0)


if __name__ == "__main__":
    main()


In [6]:
from networkx import connected_watts_strogatz_graph
from csv import writer


def main():
    # create watts strogatz network with 4000, 7000, and 10000 nodes
    # and (10%) node degree and (0.10) probability of edges creation
    filenames = ['watts_strogatz4000.csv', 'watts_strogatz7000.csv', 'watts_strogatz10000.csv']
    networksizes = [4000, 7000, 10000]

    for idx, filename in enumerate(filenames):
        graph = connected_watts_strogatz_graph(networksizes[idx], int(networksizes[idx] * 0.10), 0.10)
        try:
            with open(filename, 'w', newline='') as outfile:
                csvwriter = writer(outfile)
                for edge in graph.edges:
                    csvwriter.writerow(edge)
        except IOError as e:
            print("Unable to open or write " + filename + " due to IOError: " + e.strerror)
            exit(0)


if __name__ == "__main__":
    main()


In [ ]:
from optparse import OptionParser
from csv import reader
from networkx import Graph
from random import choice, random
from csv import writer


def write_to_csv_file(filename, rows):
    try:
        with open(filename, 'w', newline='') as outfile:
            # write to the file with outfile.write
            csvwriter = writer(outfile)
            for row in rows:
                csvwriter.writerow(row)
    except IOError as e:
        print("Unable to write " + filename + " due to Error: " + e.strerror)


def simulate_infection(net: Graph, infector: str, infection_probability: float):
    infected = set(infector)
    rows_list = [("Round", "Infection Delta", "Total infected")]
    round_count = 0
    print('Infecting ' + str(len(net.nodes())) + ' nodes, starting from ' + infector + ':')
    while len(infected) < len(net.nodes()):
        currently_infected = len(infected)
        for node in net.neighbors(choice(tuple(infected))):
            if node not in infected:
                if random() <= infection_probability :
                    infected.add(node)
        round_count += 1
        infection_delta = len(infected) - currently_infected
        if infection_delta > 0:
            rows_list.append((round_count, infection_delta, len(infected)))

    # return the infection report
    return tuple(rows_list)

def main():
    network_file = input("Enter a network file name (csv file):\n")
    infection_probability_str =input("Enter the infection probability [0 .. 1] :\n")
    infector = input("Enter infector node:\n")

    try:
        # Convert infection probability into float
        infection_probability =float(infection_probability_str)
    except ValueError:
        print("Error: probability must be float between [0 .. 1]")
        exit(0)

    graph = Graph()
    try:
        with open(network_file, 'r') as infile:
            csv = reader(infile)
            for row in csv:
                graph.add_edge(row[0], row[1])
            if infector not in graph.nodes():
                print("the infector node must be in the set of nodes:\n" + str(graph.nodes()))
                exit(0)

            #  simulate network infection
            infection_report = simulate_infection(graph, infector, infection_probability)

            # save infection report into csv file
            outfilename = "Infection-"+infection_probability_str + "-" + network_file
            write_to_csv_file(outfilename, tuple(infection_report))

    except IOError as e:
        print("Unable to open " + network_file + " due to Error: " + e.strerror)
        exit(0)

if __name__ == "__main__":
    main()


In [ ]:

from optparse import OptionParser
from csv import reader
from networkx import Graph
from random import choice, random
from csv import writer

def write_to_csv_file(filename, rows):
    try:
        with open(filename, 'w', newline='') as outfile:
            # write to the file with outfile.write
            csvwriter = writer(outfile)
            for row in rows:
                csvwriter.writerow(row)
    except IOError as e:
        print("Unable to write " + filename + " due to Error: " + e.strerror)


def simulate_inoculation(net: Graph, infector: str,
                         infection_probability: float, inoculator: str,
                         inoculation_probability: float):

    infected = set(infector)
    inoculated = set(inoculator)
    rows_list = [("Round", "Infection Delta", "New Infections",
                  "Inoculation Delta", "Total infected", "Total inoculated")]
    round_count = 0
    print('Infecting ' + str(len(net.nodes())) + ' nodes, starting from '
          + infector + ', and inoculating from ' + inoculator + ':')
    while len(infected) > 0:
        currently_infected = len(infected)
        currently_inoculated = len(inoculated)
        new_infections = 0
        if currently_infected > 0:
            for node in net.neighbors(choice(tuple(infected))):
                if node not in infected and node not in inoculated:
                    if random() <= infection_probability:
                        infected.add(node)
                        new_infections += 1
        for node in net.neighbors(choice(tuple(inoculated))):
            if node not in inoculated:
                if random() <= inoculation_probability:
                    inoculated.add(node)
                    if node in infected:
                        infected.remove(node)
        for node in inoculated:
            if node in infected:
                infected.remove(node)
        round_count += 1
        infection_delta = str(len(infected) - currently_infected)
        inoculation_delta = str(len(inoculated) - currently_inoculated)
        if not (infection_delta == '0' and inoculation_delta == '0'):
            rows_list.append((round_count, infection_delta, new_infections,
                              inoculation_delta, len(infected), len(inoculated)))

    return tuple(rows_list)


def main():
    network_file = input("Enter a network file name (csv file):\n")
    infection_probability_str =input("Enter the infection probability [0 .. 1] :\n")
    infector = input("Enter infector node:\n")
    inoculation_probability_str =input("Enter the inoculation probability [0 .. 1] :\n")
    inoculator = input("Enter inoculator node:\n")

    try:
        # Convert probability into float
        infection_probability =float(infection_probability_str)
        inoculation_probability =float(inoculation_probability_str)
    except ValueError:
        print("Error: probability must be float between [0 .. 1]")
        exit(0)

    graph = Graph()

    try:
        with open(network_file, 'r') as infile:
            csv = reader(infile)
            for row in csv:
                graph.add_edge(row[0], row[1])

            if infector not in graph.nodes():
                print("the infector node must be in the set of nodes:\n" + str(graph.nodes()))
                exit(0)
            if inoculator not in graph.nodes():
                print("the inoculator node must be in the set of nodes:\n" + str(graph.nodes()))
                exit(0)

            #  simulate network infection
            inoculation_report = simulate_inoculation(graph, infector, infection_probability,
                                                        inoculator, inoculation_probability)

            # save inoculation_report report into csv file
            outfilename = "Inoculation-"+inoculation_probability_str + "-" + network_file
            write_to_csv_file(outfilename, tuple(inoculation_report))

    except IOError as e:
        print("Unable to open " + network_file + " due to Error: " + e.strerror)
        exit(0)


if __name__ == "__main__":
    main()


In [ ]:
import matplotlib.pyplot as plt

# Existing function to simulate inoculation
# Add plotting after simulation

def plot_inoculation_report(report):
    rounds = [row[0] for row in report[1:]]
    infection_delta = [int(row[1]) for row in report[1:]]
    new_infections = [int(row[2]) for row in report[1:]]
    inoculation_delta = [int(row[3]) for row in report[1:]]
    total_infected = [int(row[4]) for row in report[1:]]
    total_inoculated = [int(row[5]) for row in report[1:]]

    plt.figure(figsize=(16, 8))

    # Total infected over time
    plt.subplot(2, 2, 1)
    plt.plot(rounds, total_infected, marker='o', linestyle='-', label='Total Infected')
    plt.title('Total Infected Over Time')
    plt.xlabel('Round')
    plt.ylabel('Total Infected')
    plt.legend()

    # Total inoculated over time
    plt.subplot(2, 2, 2)
    plt.plot(rounds, total_inoculated, marker='o', linestyle='-', color='green', label='Total Inoculated')
    plt.title('Total Inoculated Over Time')
    plt.xlabel('Round')
    plt.ylabel('Total Inoculated')
    plt.legend()

    # New infections per round
    plt.subplot(2, 2, 3)
    plt.plot(rounds, new_infections, marker='o', linestyle='-', color='red', label='New Infections')
    plt.title('New Infections Per Round')
    plt.xlabel('Round')
    plt.ylabel('New Infections')
    plt.legend()

    # New inoculations per round
    plt.subplot(2, 2, 4)
    plt.plot(rounds, inoculation_delta, marker='o', linestyle='-', color='blue', label='New Inoculations')
    plt.title('New Inoculations Per Round')
    plt.xlabel('Round')
    plt.ylabel('New Inoculations')
    plt.legend()

    plt.tight_layout()
    plt.show()

# After simulation, call the function
# plot_inoculation_report(inoculation_report)
